# FDIC Data API Client

By 503 Team 4

Source:
https://banks.data.fdic.gov/docs/#/

## Code

Import libraries

In [ ]:
import requests
import pandas as pd
import json
import urllib.parse


Define function

In [ ]:
# set endpoint
endpoint = 'https://banks.data.fdic.gov/api/'

# function
def get_all_financial_reporting_data(endpoint, reporting_period, fields, result_limit=5000):
    """
    NC (Noninsured non-deposit commercial banks) are removed below to match the query of the FDIC site https://banks.data.fdic.gov/bankfind-suite/financialreporting
    """
    
    
    # initialize data list
    financials = []
    
    # query
    fields_encoded = urllib.parse.quote(fields, safe='')
    url = f'{endpoint}financials?filters=RISDATE:{reporting_period} AND !(BKCLASS:"NC")&fields={fields_encoded}&limit={result_limit}&sort_by=REPDTE&sort_order=ASC'
    
    # get response
    response = requests.get(url)

    if response.status_code == 200:
        # response to json
        data = response.json()

        # store response data
        financial_data = [item['data'] for item in data['data']]
        financials.extend(financial_data)

    else:
        print(f'Request failed with status code {response.status_code}')
        return None

    return pd.DataFrame(financials)


Query the data

In [ ]:
# params
reporting_period = '2022-12-31'
fields = 'REPDTE,CERT,NAME,CITY,STNAME,ASSET,DEPDOM,BKCLASS,ADDRESS,ZIP,EQ,LIAB,ESTYMD,ENDEFYMD,' # no spaces between!
result_limit = 10000

# call
all_financial_reporting_data = get_all_financial_reporting_data(endpoint, reporting_period, fields, result_limit)

# display
if all_financial_reporting_data is not None:
    all_financial_reporting_data = all_financial_reporting_data.sort_values('CERT')
    display(all_financial_reporting_data.head())
    #all_financial_reporting_data.to_csv(f'../../data/quarterly_financials/{reporting_period}_quarterly_financials.csv', index=False) # save


Check data

In [ ]:
display(all_financial_reporting_data.shape)
#column_to_check = 'ACTIVE'
#display(pd.DataFrame({'col1': all_financial_reporting_data[column_to_check].value_counts().index, 'count': all_financial_reporting_data[column_to_check].value_counts().values}))



Get the last quarter for the past 30 years

In [ ]:
today = pd.Timestamp.now().date()


# define the start and end dates of the range (inclusive)
start_date = '1992-03-31'
end_date = '2022-12-31'

# generate a list of dates within the range, with a frequency of 1 quarter
dates = pd.date_range(start=start_date, end=end_date, freq='Q')

# loop
for date in dates:

    # get end of quarter
    end_of_quarter = date.strftime('%Y-%m-%d') # format
    
    fields = 'REPDTE,CERT,NAME,CITY,STNAME,ASSET,DEPDOM,BKCLASS,ADDRESS,ZIP,EQ,LIAB,ESTYMD,ENDEFYMD,EQR,' # no spaces between!
    result_limit = 10000

    # call
    all_financial_reporting_data = get_all_financial_reporting_data(endpoint, end_of_quarter, fields, result_limit)

    # display
    if all_financial_reporting_data is not None:
        all_financial_reporting_data = all_financial_reporting_data.sort_values('CERT')
        all_financial_reporting_data.to_csv(f'../../data/quarterly_financials/{end_of_quarter}_quarterly_financials.csv', index=False)
        print(f'{end_of_quarter} done')
